In [1]:
import numpy as np
import random
from tqdm import tqdm_notebook

In [2]:
# Input Class: List of Photos
class photo():
    
    def __init__(self, photoID, orient, tagsNum, tags):
        
        self.photoID = photoID
        self.orient = orient
        self.tags = tags
        self.tagsNum = tagsNum

# Output Class: List of Slides
class slide():
    
    def __init__(self, slideType, photos):
        
        self.slideType = slideType
        self.photos = photos
        
        self.tags = set([])
        
        for photo in self.photos:
            self.tags = self.tags.union(photo.tags)

def calcTransScores(slide1, slide2):
    
    numCommon = len(slide1.tags.intersection(slide2.tags))
    numDiff1 = len(slide1.tags.difference(slide2.tags))
    numDiff2 = len(slide2.tags.difference(slide1.tags))
    
    return min(numCommon, numDiff1, numDiff2)

def calcSlideshowScores(listSlides):
    
    prevSlide = None
    currSlide = None
    
    totalScore = 0
    for s in listSlides:
        prevSlide = currSlide
        currSlide = s
        
        if prevSlide == None:
            continue
        
        totalScore += calcTransScores(prevSlide, currSlide)
    
    return totalScore

In [3]:
def num_of_photos(filename):
   lines = open(filename).readlines()
   N = int(lines[0].split()[0])

   return N

def input_parser(filepath):
    lines = open(filepath).readlines()
    i = 0
    photos = []
    alltags = []
    for line in lines:
        if i > 0:
            photo_info = line.split(' ')
            orientation = photo_info[0]
            tag_num = int(photo_info[1])
            tags = []
            for j in range(tag_num):
                tags.append(photo_info[2+j].rstrip())
                alltags.extend(tags)
            photo_id = i - 1
            photos.append(photo(photoID=photo_id, orient=orientation, tagsNum = tag_num, tags = tags))
        i += 1
        
    return photos, alltags

In [4]:
def output_slideshow(filename, slide_list):
    """
    returns: Output File given a list of slide class
    """
    """Writes an output file with the required format."""
    with open(filename, 'w') as f:

        f.write(f"{len(slide_list)}\n")

        for slide in slide_list:
            if slide.slideType == 'V':
                V1, V2 = slide.photos
                f.write(f"{V1.photoID} {V2.photoID}\n")
            else:
                H = slide.photos[0]
                f.write(f"{H.photoID}\n")

In [5]:
def filterPhotos(photos, orient):
    out = []
    for photo in photos:
        #print(photo.tagsNum)
        if (photo.orient == orient):
            out.append(photo)
    
    return out

In [15]:
def generateSlides(Deck):

    N = len(Deck)
    pbar = tqdm_notebook(total = N)    
    
    myList = list(range(0,N))
    out = []
    
    while len(myList) > 0:
        N = random.choice(myList)
        myList.remove(N)
        S = createSlide([Deck[N]])
        out.append(S)
        pbar.update(1)
        
    pbar.close()
    return out

In [80]:
def generateSlides2(Deck):

    Num = len(Deck)
    TopN = int(0.25 * Num)
    pbar = tqdm_notebook(total = Num)
    
    out = []
    S1 = createSlide([Deck[0]])
    del Deck[0]
    out.append(S1)
    
    while len(Deck) > 1:
        pbar.update(1)
        #print(len(Deck))
        
        N = 0
        S2, N, Score = findNextBest(S1, Deck[0:TopN])
        
        if Score != 0:
            out.append(S2)
            del Deck[N]
            S1 = S2
        else:
            S1 = createSlide([Deck[0]])
            del Deck[0]
            out.append(S1)

    out.append(createSlide([Deck[0]]))
    
    pbar.close()
    return out

In [192]:
def findNextBest(Current, PhotoList):
    
    N = len(PhotoList)
    Score = 0
    Best = Current
    Num = 0
    
    for i in range(0,N):
        TestSlide = createSlide([PhotoList[i]])
        TestScore = calcTransScores(Current, TestSlide)
        if (TestScore > Score):
            Best = TestSlide
            Score = TestScore
            Num = i

    return Best, Num, Score

In [82]:
def createSlide(photos):
    if len(photos) == 1:
        out = slide('H', photos)
    else:
        out = slide('V', photos)
    return out

In [83]:
def get_jaccard_sim(L1, L2): 
    a = set(L1) 
    b = set(L2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))
    

In [94]:
def comparePhoto(P1, P2):
    return get_jaccard_sim(P1.tags, P2.tags)

In [468]:
def genTransition(SlideList, track=False, limit=1000):
    
    if track:
        pbar = tqdm_notebook(total = len(SlideList))
    
    F = random.randint(0, 100)
    current = SlideList[F]
    del SlideList[F]
    
    out = []
    out.append(current)
    
    while len(SlideList) > 1:
        if track:
            pbar.update(1)
        
        if len(SlideList) < limit:
            limit = len(SlideList)
            
        nextS, N, Score = findNextBestSlide(current, SlideList[0:limit])
        #print(next,N,Score)
        
        if Score > 0:
            out.append(nextS)
            del SlideList[N]
            current = nextS
        else:
            F = random.randint(0,0)
            current = SlideList[F]
            del SlideList[F]
            out.append(current)
            
    out.append(SlideList[0])

    if track:
        pbar.close()    
    return out
    

In [252]:
def findNextBestSlide(CurrentSlide, SlideList):
    
    N = len(SlideList)
    Score = 0
    BestSlide = CurrentSlide
    Num = 0
    
    for i in range(0,N):
        TestScore = calcTransScores(CurrentSlide, SlideList[i])
        if (TestScore > Score):
            BestSlide = SlideList[i]
            Score = TestScore
            Num = i

    return BestSlide, Num, Score

In [417]:
def prepareVSlides(PhotoList):
    
    count = len(PhotoList)
    breakcnt = 0
    out = []
    
    while len(PhotoList) > 0:
        found = False 
        P1 = random.choice(PhotoList)
        P2 = random.choice(PhotoList)
        similarity = comparePhoto(P1, P2)
        
        if similarity == 0:
            S = createSlide([P1,P2])
            PhotoList.remove(P1)
            PhotoList.remove(P2)
            count -= 2
            out.append(S)    
            found = True
            breakcnt = 0
            
        if found == False:
            breakcnt += 1
            
        if breakcnt == 1000:
            break
            
#    print('Start Anyhow')
#    print(len(PhotoList))
    
    while len(PhotoList) > 0:
        P1 = random.choice(PhotoList)
        P2 = random.choice(PhotoList)
        
        if P1 != P2:
            S = createSlide([P1,P2])
            PhotoList.remove(P1)
            PhotoList.remove(P2)
            out.append(S)    
        
    return out

In [277]:
def prepareHSlides(PhotoList):

    out = []
    for P in PhotoList:
        out.append(createSlide([P]))
        
    return out

In [469]:
file = 'b_lovely_landscapes'

inputfile = '../data/'+ file +'.txt'
outputfile = '../data/'+ file +'_out.txt'

N = num_of_photos(inputfile)
myPhotos, allTags = input_parser(inputfile)

In [470]:
V = filterPhotos(myPhotos, "V")
H = filterPhotos(myPhotos, "H")

In [471]:
H.sort(key=lambda x: x.tagsNum, reverse=True)
V.sort(key=lambda x: x.tagsNum, reverse=True)

In [472]:
print(len(H))
print(len(V))

80000
0


In [473]:
HS = prepareHSlides(H.copy())

In [474]:
VS = prepareVSlides(V.copy())

In [475]:
print(len(HS))
print(len(VS))
AllS = HS + VS
print(len(AllS))

80000
0
80000


In [478]:
myFinal = genTransition(AllS.copy(), track=True, limit=2000)

In [479]:
print(calcSlideshowScores(myFinal))

31998


In [398]:
output_slideshow(outputfile,myFinal)

In [450]:
myScore = 0
myCount = 10

pbar = tqdm_notebook(total = myCount)

file = 'b_lovely_landscapes'

inputfile = '../data/'+ file +'.txt'
outputfile = '../data/'+ file +'_out.txt'

N = num_of_photos(inputfile)
myPhotos, allTags = input_parser(inputfile)

V = filterPhotos(myPhotos, "V")
H = filterPhotos(myPhotos, "H")

H.sort(key=lambda x: x.tagsNum, reverse=True)
V.sort(key=lambda x: x.tagsNum, reverse=True)


for i in range(0,myCount):
    pbar.update(1)

    HS = prepareHSlides(H.copy())
    VS = prepareVSlides(V.copy())

    AllS = HS + VS
    
    myFinal = genTransition(AllS.copy())

    if calcSlideshowScores(myFinal) > myScore: 
        myScore = calcSlideshowScores(myFinal)
        output_slideshow(outputfile,myFinal)
        print("Better Score Found in {} - {}".format(i,myScore)) 

pbar.close()

KeyboardInterrupt: 

In [261]:
output_slideshow(outputfile,myFinal)

In [146]:
def groupTag(PhotoList, Num):
    
    selected = []
    remaining = []
    myList = PhotoList.copy()
    iterList = iter(myList[0:10])

    
    while True:
        try:
            photo = next(iterList)
            print(photo.photoID)
            selected.append(photo)
            myList.remove(photo)

            while len(selected) < Num:
                newphoto = random.choice(myList[10:])
                myList.remove(newphoto)
                similarity = comparePhoto(photo, newphoto)

                if (similarity > 0.1 and similarity < 0.9):
                    selected.append(newphoto)
                    print(photo.photoID, newphoto.photoID, comparePhoto(photo, newphoto))
                    
                if len(myList) == 0:
                    break
             
            if len(selected) == Num:
                remaining == myList
                break
                    
        except StopIteration:
            break
            
    return selected, remaining
    

In [ ]:
import collections
allTagsCount = collections.Counter(allTags)
sorted(allTagsCount.items(), key=lambda x: x[1], reverse=True)

In [ ]:
allTagsCount

In [ ]:
L1 = H[100].tags
L2 = H[101].tags

print(sorted(L1))
print(sorted(L2))

get_jaccard_sim(L1,L2)

In [ ]:
H = filterPhotos(myPhotos, "H")

In [ ]:
myFinal

In [ ]:
myPhoto1 = photo('0','H',['A','B'])
myPhoto2 = photo('1','V',['B','C'])
myPhoto3 = photo('2','V',['C','D'])

mySlideList = [mySlide1,mySlide2]

In [ ]:
myPhotos[0].orient

In [ ]:
T = getAllTags(myPhotos)

In [ ]:
np

mySlide1 = createSlide([H[3]])

mySlide2 = createSlide([V[2],V[5]])


calcTransScores(mySlide1,mySlide2)

In [ ]:
def generateNext(photos, myList, myLast):
    
    myScore = 0
    myNext = []
    acceptScore = 2
    myTryList = myList.copy()
    
    while myScore < acceptScore:
        N = random.choice(myTryList)
        myTryList.remove(N)
        
        Last_S = createSlide([photos[myLast]])
        Next_S = createSlide([photos[N]])
        
        myScore = calcTransScores(Last_S, Next_S)
        
        if  myScore > acceptScore - 1:
            myNext.append(Next_S)
            myList.remove(N)
            break
        
        if len(myTryList) == 0:
            break
            
    return myNext, myList, N 


def generateFirst2(photos):
    
    myList = list(range(0,len(photos)))
    myScore = 0
    myFirst2 = []
    acceptScore = 3
    
    while myScore < acceptScore:
        P1 = random.choice(myList)
        P2 = random.choice(myList)
        while P1 == P2:
            P2 = random.choice(myList)
        S1 = createSlide([photos[P1]])
        S2 = createSlide([photos[P2]])
    
        myScore = calcTransScores(S1, S2)
        
        if  myScore > acceptScore - 1:
            myFirst2.append(S1)
            myFirst2.append(S2)
            myList.remove(P1)
            myList.remove(P2)
                        
    return myFirst2, myList, P1, P2
        

In [ ]:
First2, myRemaining, myFirst, myLast = generateFirst2(H)


In [ ]:
type(First2)

In [ ]:
myNext, myRemaining, myLast = generateNext(H, myRemaining, myLast)

In [ ]:
myFinal = generateSlides(H)

In [ ]:
myFinal

In [ ]:
myFinal = list(filter(None, myFinal)) # fastest

In [ ]:
output_slideshow('../data/outputTest.txt',myFinal)